In [ ]:
# We clone all the code and tools from GitHub to this Notebook
!git clone -b 2-vdb https://github.com/kasparvonbeelen/heritageweaver.git
!sh create_env.sh

In [ ]:
# we need pillow version of 10.2.0
# we will uninstall the older version first
#!pip uninstall -y Pillow

In [ ]:
# %cd /content/heritageweaver/

In [ ]:
# %pip install -qr requirements.txt

In [ ]:
# # we need to restart the session
# # to load the updated Pillow version
# import os
# os.kill(os.getpid(), 9)

In [ ]:
# import PIL
# print(PIL.__version__)
# # this should print 10.2.0. If it doesn't, then restart your runtime:
# # Menu > Runtime > Restart session

In [ ]:
# # download data database and some additional metadata
# !gdown 1ZKZhmjVDO2U2cYRmCvu_phGgQAFY7__R
# !gdown 1BkIfIxksVjXHeWYj2JM-dYR6Ibprz8SN

In [ ]:
# # unzip archive with vector database
# !unzip -qq ce_comms_db.zip

In [1]:
import chromadb, random, requests
import ipyannotations.generic
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image
from heritageweaver.weavingtools.annotation_tools import *
from heritageweaver.weavingtools.weaving_tools import *
out_path = Path('/content/annotations')
out_path.mkdir(exist_ok=True)
sns.set()

ModuleNotFoundError: No module named 'heritageweaver'

In [ ]:
#from urllib import request
def plot_record_pair(record_pair):
    fig, axes = plt.subplots(1, 2, figsize=(15, 7.5))
    
    for i in range(2):
        record = collection_df[collection_df.record_id==record_pair[i]]
        img_path = record.img_url.values[0]
        description = soft_wrap_text(record.description.values[0])
        img = Image.open(requests.get(img_path,  stream=True).raw,).convert('RGB')
        axes[i].imshow(img)
        axes[i].set_title(description, fontsize = 18)
        axes[i].axis('off')
         
    plt.show()

In [ ]:
annotator = 'KB'
num_annotations = 10
coll1, coll2 = 'smg','nms'
modality1, modality2 = 'image', 'image'
percentile = 99.5
randomize = True

In [ ]:
collection_df = load_db("ce_comms_db",'google/siglip-base-patch16-224')
collection_df = pd.read_csv('communications_data.csv')
edges, image_similarities, inputs = get_edges(collection_db,coll1,coll2, modality1, modality2, percentile); len(edges)

In [ ]:
if randomize:
    random.shuffle(edges)
img_pairs = edges[:num_annotations]

labels = []

widget = ipyannotations.generic.ClassLabeller(
        options=['link', 'no link'], allow_freetext=True,
        display_function=plot_record_pair)


def store_annotations(entity_annotation):
    labels.append(entity_annotation)
    try:
        widget.display(img_pairs.pop(0))
    except IndexError:
        print("Finished.")
widget.on_submit(store_annotations)
widget.display(img_pairs.pop(0))
widget


In [ ]:
filter_smg = {
    "$and": [
        {
            "input_modality": {
                "$eq": modality1
            }
        },
        {
            "collection": {
                "$eq" : coll1
            }
        }
    ]
}


smg_data = collection_db.get(
                where=filter_smg,
                include=['embeddings','metadatas']

                    )

filter_nms = {
    "$and": [
        {
            "input_modality": {
                "$eq": modality2
            }
        },
        {
            "collection": {
                "$eq" : coll2
            }
        }
    ]
}

nms_data = collection_db.get(
                where=filter_nms,
                include=['embeddings','metadatas']
            )

In [ ]:
smg_ids = smg_data['ids']
nms_ids = nms_data['ids']

smg_rids = [record['record_id'] for record in smg_data['metadatas']]
nms_rids = [record['record_id'] for record in nms_data['metadatas']]

smg_emb = np.matrix(smg_data['embeddings'])
nms_emb = np.matrix(nms_data['embeddings'])
image_similarities = 1 - sp.distance.cdist(smg_emb,nms_emb, 'cosine')

threshold = np.percentile(image_similarities.reshape(-1), percentile) ; threshold

image_similarities[image_similarities >= threshold] = 1
image_similarities[image_similarities < threshold] = 0

sns.set(rc={'figure.figsize':(15.7,12.27)})

sns.heatmap(image_similarities, cmap= sns.cm.rocket_r)

In [ ]:
smg_node_ids = list(np.where(image_similarities.sum(axis=1) > 0)[0])
nms_node_ids = list(np.where(image_similarities.sum(axis=0) > 0)[0])

smg_mapping = {i:j for i,j in zip(range(len(smg_data['embeddings'])),smg_rids)}
nms_mapping = {i:j for i,j in zip(range(len(nms_data['embeddings'])),nms_rids)}

In [ ]:
edges = [(smg_mapping[i],nms_mapping[j]) for i,j in zip(*np.where(image_similarities > 0))]

In [ ]:
to_annotate = edges[:num_annotations]

In [ ]:
if randomize:
    random.shuffle(edges)
img_pairs = edges[:num_annotations]

In [ ]:
labels = []

widget = ipyannotations.generic.ClassLabeller(
        options=['link', 'no link'], allow_freetext=True,
        display_function=plot_record_pair)


def store_annotations(entity_annotation):
    labels.append(entity_annotation)
    try:
        widget.display(img_pairs.pop(0))
    except IndexError:
        print("Finished.")
widget.on_submit(store_annotations)
widget.display(img_pairs.pop(0))
widget


In [ ]:
annotations_df = pd.DataFrame(to_annotate, columns=['coll1','coll2'])
annotations_df['labels'] = labels[:num_annotations]